# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [ ]:
import requests
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm
import time
import json

pd.set_option('display.max_columns', None)
#pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 100)

In [ ]:
#!pip install geopy
#!pip install geocoder

## Список объектов

In [ ]:
all_obj_ids = []

for off_ in range(0,15000,1001):
    offset_ = off_
    limit_ = 1000
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/\
%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&\
limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]

    all_obj_ids.extend(objids)


# Данные по определенному объекту

In [ ]:
from random import sample

temp = sample(all_obj_ids, 1500)

In [ ]:
df = pd.DataFrame()

for j in tqdm(temp): #all_obj_ids

    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'+str(j)
    res = requests.get(url)
    df = pd.concat([df, pd.json_normalize(res.json()['data'])])
    time.sleep(0.5) #кто понял жизнь тот не спешит

df = df.reset_index(drop=True)


In [ ]:
locations = []
for la, lo in tqdm(zip(list(df['objLkLatitude']), list(df['objLkLongitude']))):
    geolocator = Nominatim(user_agent="http")
    location = geolocator.reverse(f"{la}, {lo}")
    try:
        locations.append(location.raw['address']['state'])
    except:
        locations.append('Not Detected')
    time.sleep(0.25)

df['location'] = locations

In [ ]:
#df.to_excel('/Users/naensamble/Desktop/data.xlsx', index=False)

In [ ]:
#сохранить в Pickle

In [ ]:
#сохранить в БД

In [ ]:
df = df.astype({'objFlatSq':'float'})
df = df.groupby('location').agg({
    'objPriceAvg':'mean',
    'objElemLivingCnt':'sum',
    'pdId':'nunique',
    'objFlatSq':'sum'})\
    .reset_index().rename(columns = {
        'objPriceAvg':'Средняя цена за кв.метр',
        'objElemLivingCnt':'Количество квартир',
        'pdId':'Количество строящихся объектов',
        'objFlatSq':'Общая площадь квартир'
    })


df.sample(5)

In [ ]:
df[df['objFlatSq'] == '0.011926.882815.08']

# Задание 3 *


In [ ]:
import requests
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm
import time
import json
from datetime import date, datetime

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 100)


def collect_objects_list():
    
    all_obj_ids = []

    for off_ in range(0,15000,1001):
        offset_ = off_
        limit_ = 1000
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/\
%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&\
limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

        res = requests.get(url)
        objects_data = res.json()
        objects_list = objects_data.get('data').get('list')
        objids = [x.get('objId') for x in objects_list]

        all_obj_ids.extend(objids)
        
    return all_obj_ids

def collect_objects():
    
    df = pd.DataFrame()

    for j in tqdm(collect_objects_list()):

        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'+str(j)
        res = requests.get(url)
        df = pd.concat([df, pd.json_normalize(res.json()['data'])])
        time.sleep(0.5) #кто понял жизнь тот не спешит

    df = df.reset_index(drop=True)
    
    return df


def append_geo():
    
    df = collect_objects()
    locations = []
    for la, lo in tqdm(zip(list(df['objLkLatitude']), list(df['objLkLongitude']))):
        geolocator = Nominatim(user_agent="http")
        location = geolocator.reverse(f"{la}, {lo}")
        try:
            locations.append(location.raw['address']['state'])
        except:
            locations.append('Not Detected')
        time.sleep(0.25)

    df['location'] = locations
    
    return df


def main():
    
    df = df.astype({'objFlatSq':'float'})
    df = df.groupby('location').agg({
        'objPriceAvg':'mean',
        'objElemLivingCnt':'sum',
        'pdId':'nunique',
        'objFlatSq':'sum'})\
        .reset_index().rename(columns = {
            'objPriceAvg':'Средняя цена за кв.метр',
            'objElemLivingCnt':'Количество квартир',
            'pdId':'Количество строящихся объектов',
            'objFlatSq':'Общая площадь квартир'
        })
    df['insert_date'] = date.today().strftime('%Y-%m-%d')
    
    return df

if __name__ == '__main__':
    main()